In [ ]:
! pip install trec-car-tools transformers
import torch
from torch.nn import functional, CosineEmbeddingLoss, BCEWithLogitsLoss, CosineSimilarity, BCELoss
from torch.utils.data.dataset import Dataset
from trec_car import read_data
from cbor import *
from transformers import BertModel, BertTokenizer, BertForNextSentencePrediction, Trainer, TrainingArguments
import random
model = BertModel.from_pretrained('bert-base-uncased')
nspmodel = BertForNextSentencePrediction.from_pretrained('bert-base-uncased') #output_hidden_states=True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
file = 'train.pages.cbor'

     |████████████████████████████████| 2.1MB 7.4MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 
     |████████████████████████████████| 3.3MB 41.6MB/s 
  Created wheel for cbor: filename=cbor-1.0.0-cp37-cp37m-linux_x86_64.whl size=51269 sha256=602c0b9e270b2bf8aeb43fd0ad51a99673a3e852942ce1b78fa4cc7595d94962
  Stored in directory: /root/.cache/pip/wheels/40/5c/a5/e6d629446a6a687ba328c55f1589234c29b99ef35b1a65dbaa
Successfully built cbor


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def generate_wrong_next(corpus):
  # helper function
  # given an array of paragraphs, generate wrong next paras
  # cant do completely random as there is a random chance to pick next para using shuffle()
  # which i want to avoid
  wrong_next = []
  second_sentences = corpus[1:len(corpus)]
  for i in range(len(second_sentences)):
    temp = second_sentences.copy()
    temp.pop(i)
    next = random.choice(temp)
    wrong_next.append(next)
  return wrong_next

In [ ]:
labels = []
first_input_ids = torch.empty(0)
second_input_ids = torch.empty(0)
pages = read_data.iter_pages(open(file, 'rb'))
count = 0
pagecount = 0
for page in pages:
      pagecount += 1
      corpus = page.get_text().split('\n')
      count += len(corpus)

      first_sentences = corpus[0:len(corpus) - 1]
      second_sentences = corpus[1:len(corpus)]
      wrong_next_sentences = generate_wrong_next(corpus)

      first_encoding = tokenizer(first_sentences, return_tensors='pt', padding='max_length', max_length=256, truncation=True)
      second_encoding = tokenizer(second_sentences, return_tensors='pt', padding='max_length', max_length=256, truncation=True)
      wrong_encoding = tokenizer(wrong_next_sentences, return_tensors='pt', padding='max_length', max_length=256, truncation=True)

      p_labels = [1.] * first_encoding['input_ids'].size()[0]
      n_labels = [-1.] * first_encoding['input_ids'].size()[0]
      labels += p_labels + n_labels

      # good = torch.tensor(first_encoding, second_encoding)
      # bad = torch.tensor(first_encoding, wrong_encoding)

      p_encodings = torch.cat((first_encoding['input_ids'], first_encoding['input_ids']), 0)
      n_encodings = torch.cat((second_encoding['input_ids'], wrong_encoding['input_ids']), 0)


      # encoding_cat = torch.cat((p_encoding['input_ids'], n_encoding['input_ids']), 0)
      first_input_ids = torch.cat((p_encodings, first_input_ids), 0)
      second_input_ids = torch.cat((n_encodings, second_input_ids), 0)

print(first_input_ids.size())
print(second_input_ids.size())
print(len(labels))
print(count) # number of paragraphs
print(pagecount)

torch.Size([11858, 256])
torch.Size([11858, 256])
11858
6046
117


In [ ]:
index = 40
# print(first_input_ids[index].type(torch.LongTensor))
# print(second_input_ids[index])
print(labels[index])
first_logits = model(first_input_ids[40:42].type(torch.LongTensor))[0]
second_logits = model(second_input_ids[40:42].type(torch.LongTensor))[0]

first_logits = torch.mean(first_logits, 2)
second_logits = torch.mean(second_logits, 2)

print(first_logits.size())
print(second_logits.size())

cos = CosineSimilarity()
c1 = cos(first_logits, second_logits)

loss = BCEWithLogitsLoss(reduction='none')



1.0
torch.Size([2, 256])
torch.Size([2, 256])


In [ ]:
class BaseBertDataSet(Dataset):
  """
  Generates a dataset of pairs of paragraphs.
  [paragraph, following paragraph]

  """
  def __init__(self, file):
    self.file = file
    self.labels = []
    self.first_input_ids = torch.empty(0)
    self.second_input_ids = torch.empty(0)

    pages = read_data.iter_pages(open(file, 'rb'))
    for page in pages:
      corpus = page.get_text().split('\n')

      first_sentences = corpus[0:len(corpus) - 1]
      second_sentences = corpus[1:len(corpus)]
      wrong_next_sentences = generate_wrong_next(corpus)

      first_encoding = tokenizer(first_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      second_encoding = tokenizer(second_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      wrong_encoding = tokenizer(wrong_next_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)

      p_labels = [1.] * first_encoding['input_ids'].size()[0]
      n_labels = [-1.] * first_encoding['input_ids'].size()[0]
      self.labels += p_labels + n_labels

      # good = torch.tensor(first_encoding, second_encoding)
      # bad = torch.tensor(first_encoding, wrong_encoding)

      p_encodings = torch.cat((first_encoding['input_ids'], first_encoding['input_ids']), 0)
      n_encodings = torch.cat((second_encoding['input_ids'], wrong_encoding['input_ids']), 0)


      # encoding_cat = torch.cat((p_encoding['input_ids'], n_encoding['input_ids']), 0)
      self.first_input_ids = torch.cat((p_encodings, self.first_input_ids), 0)
      self.second_input_ids = torch.cat((n_encodings, self.second_input_ids), 0)

    
  def __len__(self):
    return self.first_input_ids.size()[0]

  def __getitem__(self, index):
    # label as 1 because we want the the cosine to be close to 1 for these following sentences
    sample = {'first_input_ids': self.first_input_ids[index].type(torch.LongTensor), 'second_input_ids':  self.second_input_ids[index].type(torch.LongTensor), 'label': self.labels[index]}
    return sample


In [ ]:
class BertFNSDataSet(Dataset):
  """
  BertForNextSentence
  Generates a dataset of pairs of paragraphs.
  [paragraph, following paragraph]

  """
  def __init__(self, file):
    self.file = file
    self.labels = []
    self.input_ids = torch.empty(0)

    pages = read_data.iter_pages(open(file, 'rb'))
    for page in pages:
      corpus = page.get_text().split('\n')

      first_sentences = corpus[0:len(corpus) - 1]
      second_sentences = corpus[1:len(corpus)]
      wrong_next_sentences = generate_wrong_next(corpus)

      p_encoding = tokenizer(first_sentences, second_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      p_labels = [1] * p_encoding['input_ids'].size()[0]
      n_encoding = tokenizer(first_sentences, wrong_next_sentences, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
      n_labels = [0] * n_encoding['input_ids'].size()[0]
      self.labels += p_labels + n_labels
      encoding_cat = torch.cat((p_encoding['input_ids'], n_encoding['input_ids']), 0)
      self.input_ids = torch.cat((encoding_cat, self.input_ids), 0)

    
  def __len__(self):
    return self.input_ids.size()[0]

  def __getitem__(self, index):
    sample = {'input_ids': self.input_ids[index].type(torch.LongTensor), 'label': self.labels[index]}
    return sample


In [ ]:
class CosineSimTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    first_sentences = inputs['first_input_ids']
    second_sentences = inputs['second_input_ids']
    first_logits = model(first_sentences)[0]
    second_logits = model(second_sentences)[0]

    # taking the mean of the last layer
    first_logits = torch.mean(first_logits, 2) 
    second_logits = torch.mean(second_logits, 2) 
    # outputs = torch.cat((first_logits, second_logits), 0)
    outputs = {
        "first_logits": first_logits,
        "second_logits": second_logits
    }
    cos = CosineSimilarity()
    c = cos(first_logits, second_logits)

    lf = BCEWithLogitsLoss()
    loss = lf(c, labels)
    
    # sim = cos(first_logits, second_logits)
    return (loss, outputs) if return_outputs else loss



In [ ]:
class CosineEmbTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    first_sentences = inputs['first_input_ids']
    second_sentences = inputs['second_input_ids']
    first_logits = model(first_sentences)[0]
    second_logits = model(second_sentences)[0]

    # taking the mean of the last layer
    first_logits = torch.mean(first_logits, 2) 
    second_logits = torch.mean(second_logits, 2) 
    # outputs = torch.cat((first_logits, second_logits), 0)
    outputs = {
        "first_logits": first_logits,
        "second_logits": second_logits
    }
    cos = CosineEmbeddingLoss()
    loss = cos(first_logits, second_logits, labels)
    
    # sim = cos(first_logits, second_logits)
    return (loss, outputs) if return_outputs else loss

In [ ]:
ds = BaseBertDataSet(file)
ds_len = ds.__len__()
print(ds_len)
train_len = int(round(0.8 * ds_len))
eval_len = int(round(0.2 * ds_len))
train_dataset, eval_dataset = torch.utils.data.random_split(ds, [train_len, eval_len])
print(train_len)

11858
9486


In [ ]:
training_args = TrainingArguments(
    output_dir='./drive/MyDrive/thesis_models/bert/cosine_emb_bY1_batch16',          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    save_strategy='epoch'
)



trainer = CosineEmbTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset = train_dataset,
    eval_dataset = eval_dataset
)


In [ ]:
model.from_pretrained('./drive/MyDrive/thesis_models/bert/cosine_emb_bY1_batch16/checkpoint-593')

In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.evaluate()
# model.save_pretrained('./drive/MyDrive/thesis_models/bert')
# nspmodel.save_pretrained('./drive/MyDrive/thesis_models/bertfnsp')

{'eval_loss': 0.016463909298181534,
 'eval_mem_cpu_alloc_delta': 45899776,
 'eval_mem_cpu_peaked_delta': 55988224,
 'eval_runtime': 3889.5014,
 'eval_samples_per_second': 0.714,
 'init_mem_cpu_alloc_delta': 0,
 'init_mem_cpu_peaked_delta': 0}